## FDTree Class

In [2]:
import klepto
import shelve
import pickle
import numpy as np

from scipy.sparse import *

In [3]:
############### Load Data ##################
rating_matrix_csc = load_npz('netflix/sparse_matrix_100%.npz').tocsc()
rating_matrix_val_csc = load_npz('netflix/sparse_matrix_validation_75%.npz').tocsc()
print("file load DONE")
############################################

file load DONE


In [4]:
''' Save to file 'tree.pkl' '''
start = 0
end = int(rating_matrix_csc.shape[1] * 0.75)

## Step 1: Input Dataset

In [2]:
import numpy as np
class DecisionTreeModel:
    def __init__(self, source, depth_threshold=10, plambda=7, MSP_item=200):
        self.sMatrix = source
        self.real_item_num = source.shape[0]
        self.global_mean = 0  # global average of ratings   
        self.depth_threshold = depth_threshold
        self.plambda = plambda
        self.cur_depth = 0
        self.MSP_item = MSP_item

        #### Calculate rate of progress ####
        self.node_num = 0
        self.cur_node = 0
        for i in range(self.depth_threshold):
            self.node_num += 3 ** i

        
        #### Initiate Tree, lr_bound ####
        self.tree = list(range(1, self.sMatrix.shape[1]))
        self.split_item = []
        self.lr_bound = {'0': [[0, len(self.tree) - 1]]}
        self.global_mean = source.sum()/source.getnnz()

        #### Generate bias, sum_cur_t, sum_2_cur_t ####
        #### Generate rI, rU ####
        
        self.rU = {}        
        for dictname in range(151):
            d = shelve.open('./rU_data/'+str(dictname), protocol = pickle.HIGHEST_PROTOCOL)
            self.rU.update(d['content'])
            d.close()
        print("rU DONE")
        
#         self.biasUM = load_npz('netflix/biasUM.npz').tocsc()
#         print("biasUM DONE")
        
        Tree = klepto.archives.dir_archive('treeFile', {}, serialized=True)
        Tree.load()
        self.biasU       = Tree["biasU"]
        self.sum_cur_t   = Tree["sum_cur_t"]
        self.sum_2_cur_t = Tree["sum_2_cur_t"]
        self.sum_cntt    = Tree["sum_cntt"]
        self.rI          = Tree["rI"]
        self.item_size = len(self.rI)
        self.user_size = len(self.tree)        
        print("Initiation DONE!")

    def calculate_error(self, sumt, sumt_2, cntt):
        ''' Calculate error for one item-split in one node '''
        Error_i = np.sum(sumt_2 - (sumt ** 2) / (cntt + 1e-9))

        return Error_i

    def generate_decision_tree(self, lr_bound_for_node, chosen_id):
        '''
            sumtL: dict {
                itemid1: {'rating': sum of ratings for item 1, 'cnt': sum of users rated item 1}
                itemid2: {'rating': sum of ratings for item 2, 'cnt': sum of users rated item 2}
                ...
            }
            sumtL_2: dict {
                itemid1: sum of square ratings for item 1
                itemid2: sum of square ratings for item 2
                ...
            }
            lr_bound_for_node: list [leftind, rightind] for one node
        '''

        #### Terminate ####
        self.cur_depth += 1
        if self.cur_depth > self.depth_threshold or len(chosen_id) == self.item_size:
            return

        #### Choose Most Popular Items of This Node ####
        num_rec = np.zeros(self.real_item_num)
        for userid in self.tree[lr_bound_for_node[0]:(lr_bound_for_node[1] + 1)]:
            user_all_rating_id = np.array(list(self.rU[userid].keys()))
            num_rec[user_all_rating_id[:]] += 1
        sub_item_id = np.argsort(num_rec)[:self.MSP_item]

        #### Find optimum item to split ####
        min_sumtL, min_sumtD, min_sumtL_2, min_sumtD_2, min_sumtU, min_sumtU_2, Error = {}, {}, {}, {}, {}, {}, {}
        min_Error = "None"
        for itemid in sub_item_id:
#             print(itemid)
            if itemid in chosen_id:
                continue
            '''
                user_rating_item_in_nodet: np.array([ [uid01, rating01], [uid02, rating02], ... ])
                to find all users in node t who rates item i
            '''           
            user_rating_item_in_nodet = np.array([[userid, self.rU[userid][itemid]] for userid in
                                         self.tree[lr_bound_for_node[0]:(lr_bound_for_node[1] + 1)] if
                                         itemid in self.rU[userid]])
#             sumt = np.zeros((self.real_item_num, 3))
#             sumt_2 = np.zeros((self.real_item_num, 3))
#             cntt = np.zeros((self.real_item_num, 3))

#             lst_L = list(user_rating_item_in_nodet[user_rating_item_in_nodet[:, 1] >= 4, 0])
#             lst_D = list(user_rating_item_in_nodet[user_rating_item_in_nodet[:, 1] <= 3, 0])

#             realML = self.sMatrix[:, lst_L] - self.biasUM[:, lst_L]
#             realMD = self.sMatrix[:, lst_D] - self.biasUM[:, lst_D]
#             sumt[:, 0] = (realML).sum(axis=1).T
#             sumt_2[:, 0] = (realML.power(2)).sum(axis=1).T
#             cntt[:, 0] = self.sMatrix[:, lst_L].getnnz(axis=1).T
#             sumt[:, 1] = (realMD).sum(axis=1).T
#             sumt_2[:, 1] = (realMD.power(2)).sum(axis=1).T
#             cntt[:, 1] = self.sMatrix[:, lst_L].getnnz(axis=1).T
            
            sumt = np.zeros((self.real_item_num, 3))
            sumt_2 = np.zeros((self.real_item_num, 3))
            cntt = np.zeros((self.real_item_num, 3))
            for user in user_rating_item_in_nodet:
                ''' user_all_rating: array [ [itemid11, rating11], [itemid12, rating12], ... ] '''
                user_all_rating_id = np.array(list(self.rU[user[0]].keys()))
                user_all_rating = np.array(list(self.rU[user[0]].values()))
                #### calculate sumtL for node LIKE ####
                if user[1] >= 4:
                    sumt[user_all_rating_id[:], 0] += user_all_rating[:] - self.biasU[user[0]]
                    sumt_2[user_all_rating_id[:], 0] += (user_all_rating[:] - self.biasU[user[0]]) ** 2
                    cntt[user_all_rating_id[:], 0] += 1
                #### calculate sumtD for node DISLIKE ####
                elif user[1] <= 3:
                    sumt[user_all_rating_id[:], 1] += user_all_rating[:] - self.biasU[user[0]]
                    sumt_2[user_all_rating_id[:], 1] += (user_all_rating[:] - self.biasU[user[0]]) ** 2
                    cntt[user_all_rating_id[:], 1] += 1

            #### calculate sumtU for node UNKNOWN ####
            sumt[:, 2] = self.sum_cur_t[:] - sumt[:, 0] - sumt[:, 1]
            sumt_2[:, 2] = self.sum_2_cur_t[:] - sumt_2[:, 0] - sumt_2[:, 1]
            cntt[:, 2] = self.sum_cntt[:] - cntt[:, 0] - cntt[:, 1]
            Error[itemid] = self.calculate_error(sumt, sumt_2, cntt)

            if min_Error == "None" or Error[itemid] < min_Error:
                min_sumt = sumt
                min_sumt_2 = sumt_2
                min_cntt = cntt
                min_Error = Error[itemid]
        #### Find optimum split-item ####
        optimum_itemid = min(Error, key=Error.get)
        if len(self.split_item) == self.cur_depth - 1:
            self.split_item.append([optimum_itemid])
        else:
            self.split_item[self.cur_depth - 1].append(optimum_itemid)
        # self.split_item.setdefault(str(self.cur_depth-1), []).append(optimum_itemid)
        chosen_id.append(optimum_itemid)
        print("split item found!")
        #### sort tree ####
        self.lr_bound.setdefault(str(self.cur_depth), []).append([])  # for LIKE
        self.lr_bound[str(self.cur_depth)].append([])  # for DISLIKE
        self.lr_bound[str(self.cur_depth)].append([])  # for UNKNOWN
        listU, listL, listD = [], [], []
        for userid in self.tree[lr_bound_for_node[0]:(lr_bound_for_node[1] + 1)]:
            if optimum_itemid not in self.rU[userid]:
                listU.append(userid)
            elif self.rU[userid][optimum_itemid] >= 4:
                listL.append(userid)
            elif self.rU[userid][optimum_itemid] <= 3:
                listD.append(userid)
        self.tree[lr_bound_for_node[0]:(lr_bound_for_node[1] + 1)] = listL + listD + listU
        self.lr_bound[str(self.cur_depth)][-3] = [lr_bound_for_node[0],
                                                  lr_bound_for_node[0] + len(listL) - 1]  # for LIKE
        self.lr_bound[str(self.cur_depth)][-2] = [lr_bound_for_node[0] + len(listL),
                                                  lr_bound_for_node[0] + len(listL) + len(listD) - 1]  # for DISLIKE
        self.lr_bound[str(self.cur_depth)][-1] = [lr_bound_for_node[0] + len(listL) + len(listD),
                                                  lr_bound_for_node[0] + len(listL) + len(listD) + len(
                                                      listU) - 1]  # for UNKNOWN

        #### Generate Subtree of Node LIKE ####
        self.sum_cur_t = min_sumt[:, 0]
        self.sum_2_cur_t = min_sumt_2[:, 0]
        self.sum_cntt = min_cntt[:, 0]
        self.generate_decision_tree(self.lr_bound[str(self.cur_depth)][-3], chosen_id[:])
        self.cur_depth -= 1

        #### Generate Subtree of Node DISLIKE ####
        self.sum_cur_t = min_sumt[:, 1]
        self.sum_2_cur_t = min_sumt_2[:, 1]
        self.sum_cntt = min_cntt[:, 1]
        self.generate_decision_tree(self.lr_bound[str(self.cur_depth)][-2], chosen_id[:])
        self.cur_depth -= 1

        #### Generate Subtree of Node UNKNOWN ####
        self.sum_cur_t = min_sumt[:, 2]
        self.sum_2_cur_t = min_sumt_2[:, 2]
        self.sum_cntt = min_cntt[:, 2]
        self.generate_decision_tree(self.lr_bound[str(self.cur_depth)][-1], chosen_id[:])
        self.cur_depth -= 1

        #### Show Rating Progress ####
        for i in range(self.cur_depth - 1):
            print("┃", end="")
        print("┏", end="")
        self.cur_node += 1
        print("Current depth: " + str(self.cur_depth) + "        %.2f%%" % (100 * self.cur_node / self.node_num))

   
    def build_model(self):
        #### Construct the tree & get the prediction model ####
        self.generate_decision_tree(self.lr_bound['0'][0], [])

## Tree Init

In [7]:
dtmodel_realdata = DecisionTreeModel(rating_matrix_csc[:, start:end], depth_threshold = 10)

rU DONE
Initiation DONE!


## Build Tree

In [8]:
dtmodel_realdata.build_model()

split item found!
split item found!
split item found!
split item found!
split item found!
split item found!
split item found!
split item found!
split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.00%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.01%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.01%
┃┃┃┃┃┃┃┃┏Current depth: 9        0.01%
split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.02%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.02%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.02%
┃┃┃┃┃┃┃┃┏Current depth: 9        0.03%
split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.03%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.03%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.04%
┃┃┃┃┃┃┃┃┏Current depth: 9        0.04%
┃┃┃┃┃┃┃┏Current depth: 8        0.04%
split item found!
split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.05%
split item found!
┃┃┃

split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.47%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.47%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.48%
┃┃┃┃┃┃┃┃┏Current depth: 9        0.48%
split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.48%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.49%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.49%
┃┃┃┃┃┃┃┃┏Current depth: 9        0.49%
┃┃┃┃┃┃┃┏Current depth: 8        0.50%
split item found!
split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.50%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.50%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.51%
┃┃┃┃┃┃┃┃┏Current depth: 9        0.51%
split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.51%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.52%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.52%
┃┃┃┃┃┃┃┃┏Current depth: 9        0.52%
split item found!
split it

split item found!
split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.96%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.96%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.97%
┃┃┃┃┃┃┃┃┏Current depth: 9        0.97%
split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.97%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.98%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.98%
┃┃┃┃┃┃┃┃┏Current depth: 9        0.98%
split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.99%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.99%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.99%
┃┃┃┃┃┃┃┃┏Current depth: 9        1.00%
┃┃┃┃┃┃┃┏Current depth: 8        1.00%
split item found!
split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.00%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.01%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.01%
┃┃┃┃┃┃┃┃┏Current depth: 9    

split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.44%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.45%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.45%
┃┃┃┃┃┃┃┃┏Current depth: 9        1.45%
┃┃┃┃┃┃┃┏Current depth: 8        1.46%
split item found!
split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.46%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.46%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.47%
┃┃┃┃┃┃┃┃┏Current depth: 9        1.47%
split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.47%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.48%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.48%
┃┃┃┃┃┃┃┃┏Current depth: 9        1.48%
split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.49%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.49%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.49%
┃┃┃┃┃┃┃┃┏Current depth: 9        1.50%
┃┃┃┃┃┃┃┏

split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.92%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.92%
┃┃┃┃┃┃┃┃┏Current depth: 9        1.93%
split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.93%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.93%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.94%
┃┃┃┃┃┃┃┃┏Current depth: 9        1.94%
split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.94%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.95%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.95%
┃┃┃┃┃┃┃┃┏Current depth: 9        1.95%
┃┃┃┃┃┃┃┏Current depth: 8        1.96%
split item found!
split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.96%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.96%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.97%
┃┃┃┃┃┃┃┃┏Current depth: 9        1.97%
split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.97%
split it

...

split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        98.04%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        98.04%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        98.04%
┃┃┃┃┃┃┃┃┏Current depth: 9        98.05%
split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        98.05%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        98.05%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        98.06%
┃┃┃┃┃┃┃┃┏Current depth: 9        98.06%
┃┃┃┃┃┃┃┏Current depth: 8        98.06%
┃┃┃┃┃┃┏Current depth: 7        98.07%
split item found!
split item found!
split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        98.07%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        98.07%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        98.08%
┃┃┃┃┃┃┃┃┏Current depth: 9        98.08%
split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        98.08%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        98.09%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        

split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        98.51%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        98.51%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        98.51%
┃┃┃┃┃┃┃┃┏Current depth: 9        98.52%
┃┃┃┃┃┃┃┏Current depth: 8        98.52%
split item found!
split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        98.52%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        98.53%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        98.53%
┃┃┃┃┃┃┃┃┏Current depth: 9        98.53%
split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        98.54%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        98.54%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        98.54%
┃┃┃┃┃┃┃┃┏Current depth: 9        98.55%
split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        98.55%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        98.55%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        98.56%
┃┃┃┃┃┃┃┃┏Current depth: 9      

split item found!
split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        98.98%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        98.98%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        98.99%
┃┃┃┃┃┃┃┃┏Current depth: 9        98.99%
split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        98.99%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        99.00%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        99.00%
┃┃┃┃┃┃┃┃┏Current depth: 9        99.00%
split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        99.01%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        99.01%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        99.01%
┃┃┃┃┃┃┃┃┏Current depth: 9        99.02%
┃┃┃┃┃┃┃┏Current depth: 8        99.02%
┃┃┃┃┃┃┏Current depth: 7        99.02%
split item found!
split item found!
split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        99.03%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        99.03%
split item found!

split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        99.45%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        99.45%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        99.46%
┃┃┃┃┃┃┃┃┏Current depth: 9        99.46%
split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        99.46%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        99.47%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        99.47%
┃┃┃┃┃┃┃┃┏Current depth: 9        99.48%
┃┃┃┃┃┃┃┏Current depth: 8        99.48%
split item found!
split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        99.48%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        99.49%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        99.49%
┃┃┃┃┃┃┃┃┏Current depth: 9        99.49%
split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        99.50%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        99.50%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        99.50%
┃┃┃┃┃┃┃┃┏Current depth: 9      

split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        99.92%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        99.93%
┃┃┃┃┃┃┃┃┏Current depth: 9        99.93%
┃┃┃┃┃┃┃┏Current depth: 8        99.93%
split item found!
split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        99.94%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        99.94%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        99.94%
┃┃┃┃┃┃┃┃┏Current depth: 9        99.95%
split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        99.95%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        99.95%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        99.96%
┃┃┃┃┃┃┃┃┏Current depth: 9        99.96%
split item found!
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        99.96%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        99.97%
split item found!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        99.97%
┃┃┃┃┃┃┃┃┏Current depth: 9        99.97%
┃┃┃┃┃┃┃┏Current depth: 8        99.98%
┃┃┃┃┃┃┏Current depth: 7       

In [9]:
import klepto
Tree = klepto.archives.dir_archive('treeFile', cached=True, serialized=True)

Tree['fmps_lr_bound'] = dtmodel_realdata.lr_bound
Tree['fmps_tree'] = dtmodel_realdata.tree
Tree['fmps_split_item'] = dtmodel_realdata.split_item

Tree.dump()
Tree.clear()

In [8]:
import time
time.localtime(time.time())
time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()))

'2017-11-09 08:30:39'

In [10]:
del dtmodel_realdata

## MF part

In [4]:
from pyspark.mllib.recommendation import ALS
from pyspark.sql import SparkSession

class MatrixFactorization:
    def __init__(self, maxIter=15, regParam=0.01, rank=10):
        self.maxIter = maxIter
        self.regParam = regParam
        self.rank = rank
        self.spark = SparkSession.builder.master("local[*]").appName("Example").getOrCreate()
#         self.spark.conf.set("spark.driver.memory", "6g")  
        self.spark.conf.set("spark.executor.memory", "8g")  
        print("New SparkSession started...")

    def change_parameter(self, regParam):
        self.regParam = regParam

    def matrix_factorization(self, train_lst):
        ratings = self.spark.createDataFrame(train_lst)
        del train_lst
        print("createDataFrame DONE")
        model = ALS.train(ratings, self.rank, seed=10, \
                          iterations=self.maxIter, \
                          lambda_=self.regParam)
        print("MF DONE")
        userFeatures = sorted(model.userFeatures().collect(), key=lambda d: d[0], reverse=False)
        productFeatures = sorted(model.productFeatures().collect(), key=lambda d: d[0], reverse=False)
        userProfile = {each[0]: each[1].tolist() for each in userFeatures}
        itemProfile = {each[0]: each[1].tolist() for each in productFeatures}

        return userProfile, itemProfile

    def end(self):
        self.spark.stop()
        print("SparkSession stopped.")

## Tool Function

In [5]:
from scipy.sparse import find
val_num = rating_matrix_val_csc.getnnz(axis=None)
########################################## For Validation #############################################
def calculate_avg_rating_for_pesudo_user(pseudo_user_lst, sMatrix):

    # ret_array = np.zeros(sMatrix.shape[0])
    ret_array = np.array(sMatrix[:, pseudo_user_lst].sum(axis=1))[:,0]/(sMatrix[:, pseudo_user_lst].getnnz(axis=1)+1e-9)

    return ret_array


def pred_RMSE_for_validate_user(user_node_ind, user_profile, item_profile, val_user_list, val_item_list, sMatrix):
    print("RMSE calculation on valset qstarted.")
    RMSE = 0
    i = 0
    for userid, itemid in zip(val_user_list, val_item_list):
        if i % 50000 == 0:
            print("%.2f%%" % (100 * i / val_num))        
        i += 1
        RMSE += (sMatrix[itemid, userid] - np.dot(user_profile[user_node_ind[userid]], item_profile[itemid]))**2
    return (RMSE / len(val_user_list))**0.5

def generate_prediction_model(lr_bound, tree, rI, sMatrix, plambda_candidates, validation_set):
    ''' lr_bound: dict {
                level 0: [[left_bound, right_bound]], users' bound for one level, each ele in dictionary represents one node
                level 1: [[left_bound, right_bound], [left_bound, right_bound], [left_bound, right_bound]], 3
                level 2: ..., 9
            } (bound means index)
        plambda_candidates: {
            level 0: [clambda1, clambda2, clambda3, ...]
            level 1: [clambda1, clambda2, clambda3, ...]
            level 2: [clambda1, clambda2, clambda3, ...]
        }
        prediction_model: dict {
                level 0: { 'best_lambda': x, 'user_profile': ..., 'item_profile': ...}
                level 1: { 'best_lambda': x, 'user_profile': ..., 'item_profile': ...}
                level 2: { 'best_lambda': x, 'user_profile': ..., 'item_profile': ...}
            }
    '''
    prediction_model = {}
    
#     val_item_list = find(validation_set)[0]
#     val_user_list = find(validation_set)[1]
    user_node_ind = np.zeros(sMatrix.shape[1])                  #### notice that index is not id
    
    for level in lr_bound:
#        level = "10"
        print("level:", level)
        prediction_model.setdefault(level, {})
        train_lst = []       
        for pseudo_user_bound, userid in zip(lr_bound[level], range(len(lr_bound[level]))):
#             print(str(userid) + "/" + str(pow(3,int(level))))
            if pseudo_user_bound[0] > pseudo_user_bound[1]:
                continue
            pseudo_user_lst = tree[pseudo_user_bound[0]:(pseudo_user_bound[1] + 1)]
            pseudo_user_for_item = calculate_avg_rating_for_pesudo_user(pseudo_user_lst, sMatrix)
            train_lst += [(userid, itemid, float(pseudo_user_for_item[itemid])) \
                          for itemid in range(pseudo_user_for_item.shape[0]) if pseudo_user_for_item[itemid]]    
            #### find node index for each validation user ####
            user_node_ind[pseudo_user_lst] = userid      

        print("Rating Number of level " + level + ": " + str(len(train_lst)))
        #### Train MF and Do validation ####
#         min_RMSE = -1
#         for plambda in plambda_candidates[level]:
        MF = MatrixFactorization()
        MF.change_parameter(plambda_candidates[level])
        user_profile, item_profile = MF.matrix_factorization(train_lst)
        MF.end()   #### close MF spark session
#         RMSE = pred_RMSE_for_validate_user(user_node_ind, user_profile, item_profile, val_user_list, val_item_list, sMatrix)
#         if min_RMSE is -1 or RMSE < min_RMSE:
#         min_RMSE = RMSE
        min_user_profile, min_item_profile, min_lambda = user_profile, item_profile, plambda_candidates[level]
                
        prediction_model[level]['upro'], prediction_model[level]['ipro'], prediction_model[level]['plambda'] \
                                             = min_user_profile, min_item_profile, min_lambda
        print("level " + level + " training DONE")
    return prediction_model

## Load Tree from file

In [7]:
import klepto
import numpy as np
Tree = klepto.archives.dir_archive('treeFile', {}, serialized=True)
Tree.load()

In [9]:
plambda_candidates = {"0":0.02,
                     "1":0.02,
                     "2":0.02,
                     "3":0.02,
                     "4":0.02,
                     "5":0.02,
                     "6":0.02,
                     "7":0.02,
                     "8":0.02,
                     "9":0.02,
                     "10":0.02}
# for level in Tree["lr_bound"]:
#     plambda_candidates[level] = list(np.arange(0.001, 0.05, 0.005))

In [10]:
prediction_model = generate_prediction_model \
            (Tree['lr_bound'], \
             Tree['tree'], \
             Tree['rI'], \
             rating_matrix_csc[:, start:end], 
             plambda_candidates, 
             rating_matrix_val_csc)

level: 0
Rating Number of level 0: 17770


NameError: name 'MatrixFactorization' is not defined

In [5]:
import klepto
Tree = klepto.archives.dir_archive('treeFile', cached=True, serialized=True)
Tree['prediction_model'] = prediction_model
Tree.dump()
Tree.clear()

NameError: name 'prediction_model' is not defined

In [6]:
def RMSE(real_rating, pred_rating, rated_item):
    
    non_zeros = list(np.nonzero(real_rating)[0])
    non_zeros = list(set(non_zeros).difference(set(rated_item)))
    if len(non_zeros) is 0:
        return -1
    rmse = (np.sum((pred_rating[non_zeros].T[0]-real_rating[non_zeros])**2))
#     print(len(non_zeros))
#     print(real_rating[non_zeros])
#     print(pred_rating[non_zeros])
#     print((pred_rating[non_zeros]-real_rating[non_zeros])**2)
#     print(np.sum((pred_rating[non_zeros]-real_rating[non_zeros])**2))
#     print((pred_rating[non_zeros]-real_rating[non_zeros]))
#     print(rmse)
    return (rmse/len(non_zeros))**0.5


def predict(user_profile, item_profile):
    ''' 
        user_profile: array {
                        [k1, k2, k3, ... , kt]
                    } profile for certain user
        item_profile: dict {
                        itemid1: [k1, k2, k3, ... , kt], 
                        itemid2: [k1, k2, k3, ... , kt], 
                        itemid3: [k1, k2, k3, ... , kt], 
                    } profile for items in each node
     '''
    #### Calculate predict rating ####
    pred_rating = np.dot(item_profile, user_profile)
    
    return pred_rating

def pred_RMSE_for_new_user(split_item, rI, prediction_model, sM_testing):
    '''
        sM_testing: 30% test dataset (sparse matrix)
        split_item: list [
                level 0: [112],
                level 1: [48, 0, 79],
                level 2: [15, 0, 17, 1, 1, 1, 61, 0, 50]
                ...
            ]
        User: dict {
                    userid1: { itemid11: rating11, itemid12: rating12, ... } rating of user 1
                    userid2: { itemid21: rating21, itemid22: rating22, ... } rating of user 2
                    userid3: ...
                }
        return : rmse value (float)
    '''

    sM_testing_0_discard = sM_testing[1:,:]
    cnt_of_user = sM_testing.shape[1]
    rmse = []
    for userid in range(sM_testing.shape[1]):
        if userid % 100 == 0:
            print("%.2f%%" % (100 * userid / sM_testing.shape[1]))  
        pred_index = 0
        final_level = 0
        rated_item = []
        user_all_ratings = sM_testing[:,userid].nonzero()[0]
#         print("Step1 start:")
        attitude = []
        for level in range(len(split_item)):
            if split_item[level][pred_index] not in user_all_ratings:
                attitude.append('U')
                tmp_pred_index = 3*pred_index + 2
                if tmp_pred_index in prediction_model[str(int(level)+1)]['upro']:
                    final_level += 1
                    pred_index = tmp_pred_index
                else:
                    break
            elif sM_testing[split_item[level][pred_index], userid] >= 4:
                attitude.append('L')
                tmp_pred_index = 3*pred_index
                if tmp_pred_index in prediction_model[str(int(level)+1)]['upro']:
                    rated_item.append(split_item[level][pred_index]-1)
                    final_level += 1
                    pred_index = tmp_pred_index
                else:
                    break
            elif sM_testing[split_item[level][pred_index], userid] <= 3:
                attitude.append('D')
                tmp_pred_index = 3*pred_index + 1
                if tmp_pred_index in prediction_model[str(int(level)+1)]['upro']:
                    rated_item.append(split_item[level][pred_index]-1)
                    final_level += 1
                    pred_index = tmp_pred_index
                else:
                    break        

#         print("Step2 start:")
        pred_rating = predict(np.array(prediction_model[str(final_level)]['upro'][pred_index]), \
                                            np.array(list(prediction_model[str(final_level)]['ipro'].values()))) 
#         print("Step3 start:")
#         print(pred_rating)
#         print(sM_testing_0_discard[:, userid].toarray())
        tmp_rmse = RMSE(sM_testing_0_discard[:, userid].toarray(), pred_rating, rated_item)    
        # print(str(userid)+": ", attitude, tmp_rmse)
        if tmp_rmse != -1:
            rmse.append(tmp_rmse)
    return rmse
#######################################################################################################

In [7]:
import klepto
import numpy as np
Tree = klepto.archives.dir_archive('treeFile', {}, serialized=True)
Tree.load()

In [15]:
import pickle
import shelve
d = shelve.open("prediction_model", protocol=pickle.HIGHEST_PROTOCOL)
prediction_model = d['content']
rmse_result = pred_RMSE_for_new_user(Tree['split_item'][:10], Tree["rI"], prediction_model, rating_matrix_csc[:, end:])

0.00%
0:  ['U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U'] 1.04362894552
1:  ['U', 'U', 'U', 'U', 'U', 'U', 'L', 'U', 'U', 'U'] 1.4345409922
2:  ['U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U'] 0.894507137466
3:  ['U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U'] 1.31946956358
4:  ['U', 'D', 'D', 'L', 'U', 'U', 'D', 'U', 'L', 'L'] 1.70230483763
5:  ['U', 'U', 'U', 'D', 'U', 'L', 'D', 'U', 'U', 'U'] 1.18655000271
6:  ['L', 'U', 'U', 'U', 'U', 'U', 'D', 'D', 'L', 'L'] 1.05821130012
7:  ['L', 'D', 'D', 'L', 'U', 'D', 'U', 'D', 'U', 'D'] 1.24998755726
8:  ['U', 'U', 'U', 'U', 'D', 'U', 'U', 'L', 'U', 'U'] 1.42442779373
9:  ['L', 'L', 'U', 'D', 'L', 'L', 'U', 'L', 'L', 'L'] 1.28494117186
10:  ['U', 'U', 'U', 'D', 'L', 'L', 'U', 'D', 'L', 'U'] 1.0937290202
11:  ['U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U'] 0.956241451437
12:  ['D', 'D', 'U', 'U', 'L', 'U', 'L', 'L', 'U', 'L'] 1.57045632215
13:  ['U', 'D', 'U', 'U', 'D', 'U', 'U', 'U', 'L', 'U'] 1.0671749145
14:  ['U', 'U', 'U', 'L',

126:  ['L', 'U', 'U', 'U', 'L', 'U', 'U', 'U', 'U', 'L'] 0.89130238006
127:  ['U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U'] 0.6873233559
128:  ['U', 'U', 'U', 'U', 'U', 'L', 'U', 'U', 'U', 'U'] 1.29927885888
129:  ['U', 'L', 'U', 'L', 'U', 'D', 'U', 'U', 'D', 'U'] 1.05800344109
130:  ['U', 'U', 'U', 'L', 'U', 'U', 'U', 'D', 'U', 'U'] 1.37990133822
131:  ['U', 'U', 'U', 'D', 'D', 'L', 'D', 'D', 'D', 'U'] 1.28648256281
132:  ['U', 'D', 'D', 'U', 'U', 'D', 'U', 'L', 'L', 'D'] 0.722740697324
133:  ['U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U'] 1.53968441303
134:  ['D', 'U', 'U', 'U', 'L', 'U', 'D', 'U', 'U', 'D'] 1.86985311513
135:  ['U', 'D', 'U', 'U', 'D', 'U', 'U', 'L', 'U', 'L'] 1.13199090523
136:  ['L', 'D', 'D', 'U', 'L', 'D', 'U', 'D', 'L', 'U'] 1.88615546971
137:  ['L', 'U', 'L', 'U', 'U', 'L', 'L', 'U', 'L', 'L'] 1.58389721532
138:  ['D', 'L', 'D', 'U', 'U', 'D', 'D', 'L', 'U', 'U'] 0.978779172707
139:  ['D', 'U', 'U', 'L', 'U', 'U', 'U', 'U', 'U', 'U'] 1.36381001527
140: 

249:  ['U', 'U', 'D', 'D', 'L', 'L', 'L', 'U', 'L', 'U'] 1.14190924207
250:  ['U', 'D', 'U', 'L', 'U', 'D', 'U', 'U', 'D', 'L'] 1.23427241084
251:  ['U', 'U', 'U', 'U', 'U', 'L', 'L', 'U', 'U', 'L'] 0.960992562739
252:  ['U', 'U', 'U', 'D', 'U', 'L', 'U', 'L', 'D', 'U'] 1.06041061437
253:  ['U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U'] 0.815146702069
254:  ['D', 'U', 'U', 'D', 'U', 'U', 'D', 'U', 'L', 'U'] 1.29099426516
255:  ['U', 'U', 'U', 'U', 'L', 'D', 'U', 'L', 'L', 'U'] 0.980893112609
256:  ['U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U'] 1.19513223098
257:  ['U', 'U', 'U', 'U', 'U', 'L', 'L', 'L', 'L', 'L'] 1.05319582515
258:  ['U', 'U', 'U', 'L', 'D', 'L', 'U', 'D', 'L', 'U'] 1.39243914826
259:  ['U', 'L', 'U', 'U', 'L', 'L', 'U', 'L', 'L', 'U'] 1.16854436969
260:  ['L', 'U', 'U', 'D', 'U', 'D', 'L', 'D', 'U', 'U'] 1.4954406447
261:  ['L', 'U', 'D', 'D', 'U', 'D', 'L', 'U', 'U', 'L'] 1.12745181091
262:  ['U', 'L', 'U', 'L', 'U', 'L', 'L', 'U', 'U', 'U'] 0.643797865841
263

375:  ['L', 'U', 'L', 'D', 'D', 'U', 'U', 'U', 'U', 'L'] 1.46359724331
376:  ['U', 'U', 'U', 'D', 'L', 'U', 'U', 'U', 'U', 'L'] 0.734534525054
377:  ['L', 'U', 'U', 'L', 'L', 'L', 'D', 'U', 'D', 'U'] 1.16003682193
378:  ['D', 'U', 'U', 'U', 'D', 'L', 'U', 'U', 'L', 'U'] 1.32163663822
379:  ['U', 'U', 'U', 'U', 'U', 'U', 'L', 'L', 'L', 'U'] 1.467378525
380:  ['U', 'U', 'U', 'D', 'D', 'L', 'D', 'U', 'D', 'U'] 1.25339825166
381:  ['U', 'U', 'U', 'U', 'L', 'U', 'U', 'U', 'U', 'L'] 0.898986569528
382:  ['U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U'] 1.42491650211
383:  ['U', 'U', 'U', 'L', 'U', 'U', 'U', 'L', 'L', 'L'] 0.922888931766
384:  ['U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U'] 1.04604767496
385:  ['L', 'U', 'U', 'U', 'L', 'D', 'U', 'L', 'L', 'U'] 1.25978935371
386:  ['U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U'] 1.64075836233
387:  ['U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U'] 0.982964686407
388:  ['U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U'] 1.44745411034
389:

505:  ['U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U'] 0.834990240249
506:  ['U', 'U', 'U', 'L', 'U', 'U', 'U', 'U', 'U', 'U'] 0.97105006512
507:  ['U', 'U', 'L', 'U', 'L', 'L', 'U', 'U', 'U', 'L'] 1.01978624281
508:  ['U', 'U', 'U', 'L', 'U', 'U', 'L', 'L', 'U', 'L'] 0.997008881755
509:  ['D', 'D', 'U', 'L', 'D', 'U', 'U', 'D', 'U', 'U'] 1.5259336544
510:  ['U', 'U', 'U', 'D', 'U', 'U', 'U', 'U', 'D', 'U'] 1.05027216149
511:  ['U', 'L', 'U', 'L', 'U', 'U', 'L', 'U', 'U', 'U'] 0.730749261822
512:  ['U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'L'] 0.889155370267
513:  ['U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'L'] 1.69413192604
514:  ['U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U'] 0.976418941983
515:  ['D', 'D', 'U', 'D', 'U', 'U', 'D', 'D', 'U', 'U'] 1.04164668923
516:  ['D', 'D', 'U', 'L', 'D', 'D', 'L', 'D', 'D', 'D'] 1.00533162933
517:  ['L', 'U', 'L', 'U', 'D', 'U', 'D', 'L', 'L', 'D'] 1.07566423583
518:  ['U', 'L', 'U', 'L', 'U', 'L', 'D', 'U', 'D', 'U'] 1.45329668235
51

...

1368:  ['U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'D'] 1.1029846224
1369:  ['L', 'U', 'L', 'U', 'L', 'D', 'U', 'D', 'U', 'L'] 1.40520637325
1370:  ['U', 'U', 'L', 'D', 'L', 'U', 'U', 'L', 'L', 'U'] 1.13766716084
1371:  ['U', 'U', 'D', 'L', 'U', 'D', 'U', 'U', 'U', 'U'] 1.01858026225
1372:  ['U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U'] 0.730097839149
1373:  ['U', 'U', 'U', 'U', 'U', 'U', 'L', 'U', 'U', 'U'] 1.43207989745
1374:  ['U', 'U', 'U', 'L', 'L', 'L', 'D', 'U', 'L', 'U'] 0.947501890725
1375:  ['U', 'U', 'U', 'U', 'U', 'L', 'D', 'U', 'U', 'L'] 1.36441153944
1376:  ['U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U'] 1.20204064267
1377:  ['U', 'U', 'D', 'D', 'U', 'L', 'L', 'U', 'U', 'U'] 0.895877785745
1378:  ['L', 'U', 'U', 'U', 'L', 'L', 'U', 'D', 'U', 'U'] 1.35119550644
1379:  ['U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U'] 1.18186717271
1380:  ['D', 'L', 'L', 'U', 'L', 'U', 'L', 'D', 'U', 'L'] 0.754443840401
1381:  ['U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U'] 0.7

1493:  ['U', 'D', 'U', 'U', 'U', 'L', 'U', 'L', 'U', 'U'] 1.30076050308
1494:  ['U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U'] 0.473761245191
1495:  ['U', 'U', 'U', 'U', 'D', 'U', 'U', 'U', 'U', 'U'] 0.625453116925
1496:  ['U', 'U', 'U', 'L', 'U', 'D', 'D', 'D', 'U', 'U'] 1.66697527126
1497:  ['L', 'U', 'U', 'U', 'U', 'L', 'U', 'U', 'U', 'L'] 1.32090995678
1498:  ['L', 'U', 'L', 'L', 'U', 'U', 'U', 'L', 'L', 'L'] 1.37117235052
1499:  ['U', 'U', 'U', 'L', 'D', 'L', 'L', 'U', 'D', 'U'] 0.917539503021
1.25%
1500:  ['L', 'U', 'L', 'U', 'U', 'U', 'U', 'U', 'U', 'L'] 1.22747588085
1501:  ['L', 'L', 'U', 'L', 'U', 'L', 'L', 'L', 'U', 'U'] 0.903736342525
1502:  ['U', 'U', 'U', 'D', 'U', 'U', 'U', 'D', 'L', 'D'] 0.99913033767
1503:  ['U', 'U', 'D', 'U', 'U', 'D', 'U', 'U', 'U', 'U'] 2.27290138682
1504:  ['U', 'L', 'L', 'U', 'D', 'L', 'L', 'D', 'U', 'L'] 1.70954530358
1505:  ['L', 'U', 'L', 'L', 'U', 'D', 'L', 'D', 'U', 'D'] 1.04853364561
1506:  ['U', 'U', 'U', 'U', 'U', 'D', 'D', 'L', 'U', '

1618:  ['L', 'D', 'U', 'U', 'D', 'U', 'U', 'U', 'U', 'L'] 0.947160870836
1619:  ['U', 'U', 'U', 'D', 'U', 'U', 'U', 'U', 'U', 'L'] 1.59478559366
1620:  ['D', 'D', 'D', 'L', 'D', 'D', 'D', 'D', 'D', 'D'] 1.12693243011
1621:  ['U', 'U', 'U', 'U', 'L', 'U', 'D', 'L', 'D', 'U'] 1.62050656549
1622:  ['U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'L', 'D'] 1.07987075321
1623:  ['U', 'U', 'U', 'U', 'L', 'U', 'U', 'U', 'U', 'U'] 1.32369609742
1624:  ['L', 'L', 'D', 'U', 'L', 'U', 'U', 'L', 'D', 'L'] 1.01558331965
1625:  ['L', 'D', 'U', 'U', 'U', 'L', 'U', 'U', 'U', 'U'] 1.398230236
1626:  ['U', 'U', 'U', 'L', 'L', 'U', 'U', 'L', 'U', 'U'] 1.5247349799
1627:  ['D', 'L', 'D', 'U', 'L', 'U', 'U', 'L', 'U', 'L'] 1.35060617724
1628:  ['L', 'D', 'U', 'U', 'U', 'U', 'U', 'U', 'D', 'U'] 1.32372663101
1629:  ['U', 'U', 'U', 'D', 'U', 'U', 'U', 'U', 'U', 'U'] 1.11152349305
1630:  ['U', 'U', 'U', 'U', 'U', 'U', 'L', 'U', 'U', 'U'] 1.15423273214
1631:  ['U', 'D', 'U', 'U', 'U', 'U', 'U', 'U', 'L', 'U'] 1.130562

1747:  ['U', 'U', 'D', 'L', 'L', 'L', 'U', 'D', 'U', 'U'] 1.14916249601
1748:  ['U', 'U', 'U', 'U', 'U', 'U', 'D', 'L', 'D', 'U'] 1.42633430157
1749:  ['U', 'U', 'U', 'U', 'U', 'U', 'U', 'D', 'U', 'U'] 0.869879741637
1750:  ['U', 'U', 'U', 'L', 'U', 'U', 'D', 'U', 'U', 'D'] 0.800649194975
1751:  ['U', 'U', 'U', 'U', 'L', 'L', 'U', 'D', 'D', 'L'] 1.30943281976
1752:  ['D', 'U', 'L', 'L', 'D', 'L', 'U', 'D', 'L', 'U'] 1.06176081129
1753:  ['U', 'U', 'U', 'D', 'U', 'U', 'U', 'L', 'U', 'U'] 1.06568337939
1754:  ['U', 'U', 'U', 'D', 'D', 'L', 'D', 'D', 'L', 'U'] 1.16470276464
1755:  ['U', 'U', 'D', 'U', 'D', 'U', 'U', 'U', 'U', 'U'] 1.18178059925
1756:  ['U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U'] 0.94153657499
1757:  ['U', 'U', 'U', 'U', 'L', 'U', 'U', 'U', 'U', 'U'] 0.998932642341


KeyboardInterrupt: 

In [11]:
rmse_result

1.1872843595007572

In [19]:
Tree['lr_bound']['10'][-1]

[302844, 360140]